<a href="https://colab.research.google.com/github/lubnakhan-Github-Account/agentic_class_work/blob/main/Guardrail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Input Output Guardrail***

In [ ]:
!pip install -qU openai-agents

In [3]:
import os
from google.colab import userdata

import nest_asyncio
nest_asyncio.apply()



In [5]:
api_key=userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key



In [13]:
import asyncio
from agents import Agent,Runner,RunContextWrapper,GuardrailFunctionOutput,input_guardrail,output_guardrail
from typing import Any
from pydantic import BaseModel,Field

class GiaicAdmissionOpen(BaseModel):
  is_open:bool=Field(description="if user ask about Giaic admission  are open you set in this field true ")
  reasioning:str= Field(description="you give reasion about the admission of giaic.")

inputGuardrail_agent =Agent(
    name="inputGuardrail_agent",
    instructions="you check if user ask about open admission of giaic.",
     model="gpt-4.1-mini",
     output_type=GiaicAdmissionOpen,
)
@input_guardrail
async def openAddmition(wrapper:RunContextWrapper, agent:Agent, input:Any)-> GuardrailFunctionOutput:
  guardrailresult= await Runner.run(inputGuardrail_agent, input)

  return GuardrailFunctionOutput(
      output_info=guardrailresult.final_output,
      tripwire_triggered=guardrailresult.final_output.is_open
  )
# -------------------------------------------------------------------------------------------------

outputGuardrail_agent =Agent(
    name="outputGuardrail_agent",
    instructions="you check if user ask about open admission of giaic.",
     model="gpt-4.1-mini",
     output_type=GiaicAdmissionOpen,
)

@output_guardrail
async def openAddmitionoutput(wrapper:RunContextWrapper, agent:Agent, input:Any)-> GuardrailFunctionOutput:
  outputguardrailresult= await Runner.run(outputGuardrail_agent, input)

  return GuardrailFunctionOutput(
      output_info=outputguardrailresult.final_output,
      tripwire_triggered=outputguardrailresult.final_output.is_open
  )


main_agent= Agent(
    name="main_agent",
    instructions="you are a helpfull assistent.",
    model="gpt-4.1-mini",
    input_guardrails=[openAddmition],
    output_guardrails=[openAddmitionoutput]
)

result= Runner.run_sync(main_agent, "I want to admission in university in Dubai.if user ask about the admission of Dubai dont reply.")
print(result.final_output)

Understood! If you have questions about admission in universities other than those in Dubai, feel free to ask, and I'll be happy to help.
